# Academic Status and Dropout Prediction - Model Experimentation

This notebook focuses on developing, evaluating, and fine-tuning machine learning models for predicting academic status and dropout risk using the processed data from our feature engineering stage. We'll experiment with various algorithms, evaluate their performance, and select the best model for deployment.

## Table of Contents
1. [Setup and Configuration](#1.-Setup-and-Configuration)
2. [Loading Processed Data](#2.-Loading-Processed-Data)
3. [Baseline Models](#3.-Baseline-Models)
4. [Model Evaluation Framework](#4.-Model-Evaluation-Framework)
5. [Advanced Models](#5.-Advanced-Models)
6. [Hyperparameter Tuning](#6.-Hyperparameter-Tuning)
7. [Ensemble Methods](#7.-Ensemble-Methods)
8. [Model Interpretability](#8.-Model-Interpretability)
9. [Final Model Selection](#9.-Final-Model-Selection)
10. [Model Deployment Preparation](#10.-Model-Deployment-Preparation)

## 1. Setup and Configuration

Let's first import the necessary libraries and set up the environment.

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
import warnings
import time
from datetime import datetime

# Machine learning libraries
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Evaluation libraries
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV, RandomizedSearchCV

# Model interpretation libraries
from sklearn.inspection import permutation_importance
import shap

# Import custom utility functions if any
import sys
sys.path.append('../')
# from src.models.evaluate_model import plot_confusion_matrix  # Uncomment when available

# Configure visualizations
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set(style="whitegrid")
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.3f}'.format)

# Set random state for reproducibility
RANDOM_STATE = 42

## 2. Loading Processed Data

Let's load the processed data created during the feature engineering phase.

In [ ]:
# Define paths
processed_data_dir = '../data/processed'
models_dir = '../ml_models/trained_models'

# Create models directory if it doesn't exist
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

# Load training and testing data
train_data = pd.read_csv(f'{processed_data_dir}/train_data.csv')
test_data = pd.read_csv(f'{processed_data_dir}/test_data.csv')

print(f"Training data shape: {train_data.shape}")
print(f"Testing data shape: {test_data.shape}")

# Load label encoder
with open(f'{processed_data_dir}/label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

# Load feature information
with open(f'{processed_data_dir}/feature_info.pkl', 'rb') as f:
    feature_info = pickle.load(f)

# Extract target mapping
target_mapping = feature_info['target_mapping']
print("\nTarget class mapping:")
for original, encoded in target_mapping.items():
    print(f"{original} -> {encoded}")

# Extract features and target
X_train = train_data.drop(columns=['Target_encoded'])
y_train = train_data['Target_encoded']
X_test = test_data.drop(columns=['Target_encoded'])
y_test = test_data['Target_encoded']

# Extract feature names
feature_names = X_train.columns.tolist()

# Verify if 'Target' column exists and remove it
if 'Target' in feature_names:
    X_train = X_train.drop(columns=['Target'])
    X_test = X_test.drop(columns=['Target'])
    feature_names.remove('Target')

print(f"\nNumber of features: {len(feature_names)}")

# Check class distribution
print("\nClass distribution in training set:")
print(pd.Series(y_train).value_counts(normalize=True) * 100)

print("\nClass distribution in testing set:")
print(pd.Series(y_test).value_counts(normalize=True) * 100)

## 3. Baseline Models

Let's first establish baseline performance with simple models.

In [ ]:
# Function to evaluate model performance
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name=None):
    start_time = time.time()
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_prob = None
    
    # Get probability predictions if method exists
    if hasattr(model, "predict_proba"):
        y_pred_prob = model.predict_proba(X_test)
    
    # Calculate training time
    training_time = time.time() - start_time
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision_macro = precision_score(y_test, y_pred, average='macro')
    recall_macro = recall_score(y_test, y_pred, average='macro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    
    # ROC-AUC score for multi-class (if probability predictions available)
    roc_auc = None
    if y_pred_prob is not None:
        try:
            # For multi-class, use One-vs-Rest approach
            roc_auc = roc_auc_score(y_test, y_pred_prob, multi_class='ovr')
        except:
            # If there's an issue, skip ROC-AUC calculation
            roc_auc = None
    
    # Create evaluation results dictionary
    results = {
        'Model': model_name if model_name else model.__class__.__name__,
        'Accuracy': accuracy,
        'Precision (Macro)': precision_macro,
        'Recall (Macro)': recall_macro,
        'F1 Score (Macro)': f1_macro,
        'ROC-AUC (OvR)': roc_auc,
        'Training Time (s)': training_time
    }
    
    return results, y_pred, y_pred_prob

# Function to plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, class_names=None, figsize=(10, 8), title="Confusion Matrix"):
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=figsize)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names if class_names else 'auto',
                yticklabels=class_names if class_names else 'auto')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(title)
    plt.show()

In [ ]:
# Define baseline models
baseline_models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=RANDOM_STATE),
    'Decision Tree': DecisionTreeClassifier(random_state=RANDOM_STATE),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=RANDOM_STATE),
    'SVM': SVC(probability=True, random_state=RANDOM_STATE),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Gaussian Naive Bayes': GaussianNB(),
    'LDA': LinearDiscriminantAnalysis()
}

# Evaluate each baseline model
baseline_results = []
predictions = {}

for name, model in baseline_models.items():
    print(f"Training {name}...")
    results, y_pred, y_pred_prob = evaluate_model(model, X_train, X_test, y_train, y_test, name)
    baseline_results.append(results)
    predictions[name] = (y_pred, y_pred_prob)
    print(f"Completed {name} with accuracy: {results['Accuracy']:.4f}")

# Display results in a DataFrame
baseline_results_df = pd.DataFrame(baseline_results)
display(baseline_results_df.sort_values(by='F1 Score (Macro)', ascending=False))

In [ ]:
# Visualize baseline model performance
plt.figure(figsize=(14, 8))

# Sort by F1 score
sorted_results = baseline_results_df.sort_values(by='F1 Score (Macro)', ascending=False)
model_names = sorted_results['Model']

# Plot bar chart for different metrics
bar_width = 0.15
r1 = np.arange(len(model_names))
r2 = [x + bar_width for x in r1]
r3 = [x + bar_width for x in r2]
r4 = [x + bar_width for x in r3]

plt.bar(r1, sorted_results['Accuracy'], width=bar_width, label='Accuracy', color='skyblue')
plt.bar(r2, sorted_results['Precision (Macro)'], width=bar_width, label='Precision', color='lightgreen')
plt.bar(r3, sorted_results['Recall (Macro)'], width=bar_width, label='Recall', color='salmon')
plt.bar(r4, sorted_results['F1 Score (Macro)'], width=bar_width, label='F1 Score', color='purple')

plt.xlabel('Model', fontsize=12)
plt.ylabel('Score', fontsize=12)
plt.title('Baseline Model Performance Comparison', fontsize=16)
plt.xticks([r + bar_width*1.5 for r in range(len(model_names))], model_names, rotation=45, ha='right')
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
# Let's look at the confusion matrix for the best baseline model
best_baseline_model = sorted_results.iloc[0]['Model']
print(f"Best baseline model based on F1 Score: {best_baseline_model}")

y_pred, _ = predictions[best_baseline_model]
class_names = [label_encoder.inverse_transform([i])[0] for i in range(len(label_encoder.classes_))]

plot_confusion_matrix(y_test, y_pred, class_names=class_names, 
                      title=f"Confusion Matrix for {best_baseline_model}")

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=class_names))

## 4. Model Evaluation Framework

Let's create a more robust evaluation framework using cross-validation.

In [ ]:
# Function to perform k-fold cross-validation
def cross_validate_model(model, X, y, cv=5, scoring='f1_macro'):
    # Create stratified k-fold
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=RANDOM_STATE)
    
    # Cross-validation scores
    cv_scores = cross_val_score(model, X, y, cv=skf, scoring=scoring)
    
    return {
        'mean_score': cv_scores.mean(),
        'std_score': cv_scores.std(),
        'all_scores': cv_scores
    }

# Apply cross-validation to baseline models
cv_results = []

for name, model in baseline_models.items():
    print(f"Cross-validating {name}...")
    cv_result = cross_validate_model(model, X_train, y_train)
    cv_results.append({
        'Model': name,
        'Mean F1 Score': cv_result['mean_score'],
        'Std F1 Score': cv_result['std_score']
    })
    print(f"Completed {name} with mean F1 score: {cv_result['mean_score']:.4f} ± {cv_result['std_score']:.4f}")

# Display cross-validation results
cv_results_df = pd.DataFrame(cv_results)
display(cv_results_df.sort_values(by='Mean F1 Score', ascending=False))

# Visualize cross-validation results
plt.figure(figsize=(12, 8))
sorted_cv = cv_results_df.sort_values(by='Mean F1 Score', ascending=False)
plt.bar(sorted_cv['Model'], sorted_cv['Mean F1 Score'], yerr=sorted_cv['Std F1 Score'], 
        color='skyblue', capsize=7, alpha=0.8)
plt.xlabel('Model')
plt.ylabel('Mean F1 Score')
plt.title('Cross-Validation Results (F1 Score)')
plt.xticks(rotation=45, ha='right')
plt.ylim(0, 1.0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## 5. Advanced Models

Let's try more sophisticated algorithms like XGBoost and LightGBM.

In [ ]:
# Define advanced models
advanced_models = {
    'XGBoost': XGBClassifier(n_estimators=100, random_state=RANDOM_STATE),
    'LightGBM': LGBMClassifier(n_estimators=100, random_state=RANDOM_STATE),
    'Neural Network': MLPClassifier(max_iter=1000, random_state=RANDOM_STATE)
}

# Evaluate advanced models
advanced_results = []
advanced_predictions = {}

for name, model in advanced_models.items():
    print(f"Training {name}...")
    results, y_pred, y_pred_prob = evaluate_model(model, X_train, X_test, y_train, y_test, name)
    advanced_results.append(results)
    advanced_predictions[name] = (y_pred, y_pred_prob)
    print(f"Completed {name} with accuracy: {results['Accuracy']:.4f}")

# Display results in a DataFrame
advanced_results_df = pd.DataFrame(advanced_results)
display(advanced_results_df.sort_values(by='F1 Score (Macro)', ascending=False))

# Combine baseline and advanced results
all_results_df = pd.concat([baseline_results_df, advanced_results_df])
all_sorted = all_results_df.sort_values(by='F1 Score (Macro)', ascending=False)
display(all_sorted.head(5))

# Let's look at the confusion matrix for the best advanced model
best_advanced_model = advanced_results_df.sort_values(by='F1 Score (Macro)', ascending=False).iloc[0]['Model']
print(f"Best advanced model based on F1 Score: {best_advanced_model}")

y_pred_adv, _ = advanced_predictions[best_advanced_model]
plot_confusion_matrix(y_test, y_pred_adv, class_names=class_names, 
                      title=f"Confusion Matrix for {best_advanced_model}")

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_adv, target_names=class_names))

## 6. Hyperparameter Tuning

Let's optimize the top-performing models using hyperparameter tuning.

In [ ]:
# Select top models for hyperparameter tuning
top_models = all_sorted.head(3)['Model'].tolist()
print(f"Top models for hyperparameter tuning: {top_models}")

In [ ]:
# Function to tune hyperparameters using RandomizedSearchCV
def tune_hyperparameters(model, param_grid, X, y, cv=5, n_iter=20, scoring='f1_macro'):
    # Create randomized search
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=n_iter,
        cv=cv,
        scoring=scoring,
        n_jobs=-1,
        random_state=RANDOM_STATE,
        verbose=1
    )
    
    # Fit the search
    start_time = time.time()
    random_search.fit(X, y)
    tuning_time = time.time() - start_time
    
    # Best parameters and score
    best_params = random_search.best_params_
    best_score = random_search.best_score_
    best_estimator = random_search.best_estimator_
    
    return {
        'best_params': best_params,
        'best_score': best_score,
        'best_estimator': best_estimator,
        'tuning_time': tuning_time
    }

In [ ]:
# Define hyperparameter grids for top models
param_grids = {}

# XGBoost parameters
if 'XGBoost' in top_models:
    param_grids['XGBoost'] = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [3, 4, 5, 6, 7, 8],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
        'gamma': [0, 0.1, 0.2, 0.3],
        'min_child_weight': [1, 3, 5, 7]
    }

# Random Forest parameters
if 'Random Forest' in top_models:
    param_grids['Random Forest'] = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [None, 5, 10, 15, 20, 25],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2', None]
    }

# Gradient Boosting parameters
if 'Gradient Boosting' in top_models:
    param_grids['Gradient Boosting'] = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [3, 4, 5, 6],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

# LightGBM parameters
if 'LightGBM' in top_models:
    param_grids['LightGBM'] = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [3, 4, 5, 6, 7, 8, -1],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_samples': [5, 10, 20, 30],
        'num_leaves': [31, 50, 70, 90, 121]
    }

# Neural Network parameters
if 'Neural Network' in top_models:
    param_grids['Neural Network'] = {
        'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (100, 100)],
        'activation': ['relu', 'tanh', 'logistic'],
        'alpha': [0.0001, 0.001, 0.01, 0.1],
        'learning_rate': ['constant', 'adaptive'],
        'solver': ['adam', 'sgd'],
        'max_iter': [1000, 2000, 3000]
    }

# SVM parameters
if 'SVM' in top_models:
    param_grids['SVM'] = {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto', 0.1, 0.01, 0.001],
        'kernel': ['rbf', 'poly', 'sigmoid']
    }

# Logistic Regression parameters
if 'Logistic Regression' in top_models:
    param_grids['Logistic Regression'] = {
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        'penalty': ['l1', 'l2', 'elasticnet', None],
        'max_iter': [100, 500, 1000]
    }

In [ ]:
# Perform hyperparameter tuning for top models
tuning_results = {}
tuned_models = {}

for model_name in top_models:
    if model_name in param_grids:
        print(f"\nTuning hyperparameters for {model_name}...")
        
        # Get base model and parameter grid
        if model_name == 'XGBoost':
            base_model = XGBClassifier(random_state=RANDOM_STATE)
        elif model_name == 'Random Forest':
            base_model = RandomForestClassifier(random_state=RANDOM_STATE)
        elif model_name == 'Gradient Boosting':
            base_model = GradientBoostingClassifier(random_state=RANDOM_STATE)
        elif model_name == 'LightGBM':
            base_model = LGBMClassifier(random_state=RANDOM_STATE)
        elif model_name == 'Neural Network':
            base_model = MLPClassifier(random_state=RANDOM_STATE)
        elif model_name == 'SVM':
            base_model = SVC(probability=True, random_state=RANDOM_STATE)
        elif model_name == 'Logistic Regression':
            base_model = LogisticRegression(random_state=RANDOM_STATE)
        
        param_grid = param_grids[model_name]
        
        # Tune hyperparameters
        tuning_result = tune_hyperparameters(base_model, param_grid, X_train, y_train)
        tuning_results[model_name] = tuning_result
        
        # Save best estimator
        tuned_models[model_name] = tuning_result['best_estimator']
        
        print(f"Best F1 score for {model_name}: {tuning_result['best_score']:.4f}")
        print(f"Best parameters for {model_name}:")
        for param, value in tuning_result['best_params'].items():
            print(f"  {param}: {value}")
    else:
        print(f"No parameter grid defined for {model_name}. Skipping.")

In [ ]:
# Evaluate tuned models on test set
tuned_results = []
tuned_predictions = {}

for name, model in tuned_models.items():
    print(f"Evaluating tuned {name}...")
    results, y_pred, y_pred_prob = evaluate_model(model, X_train, X_test, y_train, y_test, f"Tuned {name}")
    tuned_results.append(results)
    tuned_predictions[name] = (y_pred, y_pred_prob)
    print(f"Completed tuned {name} with accuracy: {results['Accuracy']:.4f}")

# Display results in a DataFrame
tuned_results_df = pd.DataFrame(tuned_results)
display(tuned_results_df.sort_values(by='F1 Score (Macro)', ascending=False))

# Compare with original untuned models
comparison_models = []
for model_name in tuned_models.keys():
    # Get original results
    original_results = all_results_df[all_results_df['Model'] == model_name].to_dict('records')[0]
    original_results['Model'] = f"Original {model_name}"
    comparison_models.append(original_results)

# Combine with tuned results
comparison_df = pd.DataFrame(comparison_models + tuned_results)
display(comparison_df.sort_values(by=['Model', 'F1 Score (Macro)'], ascending=[True, False]))

# Let's look at the confusion matrix for the best tuned model
best_tuned_model_name = tuned_results_df.sort_values(by='F1 Score (Macro)', ascending=False).iloc[0]['Model']
print(f"Best tuned model based on F1 Score: {best_tuned_model_name}")

best_tuned_model = best_tuned_model_name.replace('Tuned ', '')
y_pred_tuned, _ = tuned_predictions[best_tuned_model]
plot_confusion_matrix(y_test, y_pred_tuned, class_names=class_names, 
                      title=f"Confusion Matrix for {best_tuned_model_name}")

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_tuned, target_names=class_names))

## 7. Ensemble Methods

Let's explore ensemble methods to potentially improve performance further.

In [ ]:
# Create a voting classifier using the best tuned models
voting_classifiers = []
for name, model in tuned_models.items():
    voting_classifiers.append((name, model))

# Create and evaluate the voting classifier
if len(voting_classifiers) >= 2:  # Need at least 2 classifiers for voting
    # Hard voting
    voting_hard = VotingClassifier(estimators=voting_classifiers, voting='hard')
    hard_results, hard_pred, _ = evaluate_model(voting_hard, X_train, X_test, y_train, y_test, "Voting (Hard)")
    
    # Soft voting
    voting_soft = VotingClassifier(estimators=voting_classifiers, voting='soft')
    soft_results, soft_pred, soft_pred_prob = evaluate_model(voting_soft, X_train, X_test, y_train, y_test, "Voting (Soft)")
    
    # Display voting results
    voting_results_df = pd.DataFrame([hard_results, soft_results])
    display(voting_results_df)
    
    # Compare with best individual model
    best_individual = tuned_results_df.iloc[0].to_dict()
    comparison = pd.DataFrame([best_individual, hard_results, soft_results])
    display(comparison)
    
    # Plot confusion matrix for best voting method
    if soft_results['F1 Score (Macro)'] > hard_results['F1 Score (Macro)']:
        best_voting = "Soft Voting"
        best_voting_pred = soft_pred
    else:
        best_voting = "Hard Voting"
        best_voting_pred = hard_pred
        
    plot_confusion_matrix(y_test, best_voting_pred, class_names=class_names, 
                          title=f"Confusion Matrix for {best_voting}")
    
    # Print classification report
    print(f"Classification Report for {best_voting}:")
    print(classification_report(y_test, best_voting_pred, target_names=class_names))
else:
    print("Not enough tuned models to create a voting classifier. Need at least 2.")

In [ ]:
# Create a stacking classifier using the tuned models
if len(tuned_models) >= 2:  # Need at least 2 base estimators for stacking
    # Define base estimators
    estimators = [(name, model) for name, model in tuned_models.items()]
    
    # Try different meta-classifiers
    meta_classifiers = {
        'Logistic Regression': LogisticRegression(random_state=RANDOM_STATE),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE),
        'XGBoost': XGBClassifier(n_estimators=100, random_state=RANDOM_STATE)
    }
    
    stacking_results = []
    stacking_predictions = {}
    
    for meta_name, meta_clf in meta_classifiers.items():
        print(f"Training Stacking Classifier with {meta_name} as meta-classifier...")
        
        stacking = StackingClassifier(
            estimators=estimators,
            final_estimator=meta_clf,
            cv=5,
            stack_method='predict_proba'
        )
        
        results, y_pred, y_pred_prob = evaluate_model(stacking, X_train, X_test, y_train, y_test, 
                                                      f"Stacking ({meta_name})")
        stacking_results.append(results)
        stacking_predictions[meta_name] = (y_pred, y_pred_prob)
        print(f"Completed Stacking ({meta_name}) with accuracy: {results['Accuracy']:.4f}")
    
    # Display stacking results
    stacking_results_df = pd.DataFrame(stacking_results)
    display(stacking_results_df.sort_values(by='F1 Score (Macro)', ascending=False))
    
    # Compare with best individual model and voting
    best_meta = stacking_results_df.sort_values(by='F1 Score (Macro)', ascending=False).iloc[0]['Model']
    best_meta_name = best_meta.replace('Stacking (', '').replace(')', '')
    
    # Extract best predictions
    y_pred_stacking, _ = stacking_predictions[best_meta_name]
    
    # Plot confusion matrix
    plot_confusion_matrix(y_test, y_pred_stacking, class_names=class_names, 
                          title=f"Confusion Matrix for {best_meta}")
    
    # Print classification report
    print(f"Classification Report for {best_meta}:")
    print(classification_report(y_test, y_pred_stacking, target_names=class_names))
else:
    print("Not enough tuned models to create a stacking classifier. Need at least 2.")

## 8. Model Interpretability

Let's analyze feature importance and gain insights from our best model.

In [ ]:
# Determine the best overall model based on performance
all_model_results = pd.concat([baseline_results_df, advanced_results_df, tuned_results_df])
if 'voting_results_df' in locals():
    all_model_results = pd.concat([all_model_results, voting_results_df])
if 'stacking_results_df' in locals():
    all_model_results = pd.concat([all_model_results, stacking_results_df])

best_overall = all_model_results.sort_values(by='F1 Score (Macro)', ascending=False).iloc[0]
best_model_name = best_overall['Model']
print(f"Best overall model: {best_model_name} with F1 Score: {best_overall['F1 Score (Macro)']:.4f}")

# Get the best model object for interpretation
if best_model_name.startswith('Tuned '):
    model_key = best_model_name.replace('Tuned ', '')
    if model_key in tuned_models:
        best_model = tuned_models[model_key]
    else:
        print(f"Model {model_key} not found in tuned_models. Using the first tuned model.")
        best_model = list(tuned_models.values())[0]
elif best_model_name in baseline_models:
    best_model = baseline_models[best_model_name]
elif best_model_name in advanced_models:
    best_model = advanced_models[best_model_name]
elif best_model_name.startswith('Voting'):
    if 'voting_soft' in locals() and best_model_name == "Voting (Soft)":
        best_model = voting_soft
    elif 'voting_hard' in locals() and best_model_name == "Voting (Hard)":
        best_model = voting_hard
    else:
        print(f"Voting model {best_model_name} not found. Using first tuned model.")
        best_model = list(tuned_models.values())[0]
elif best_model_name.startswith('Stacking'):
    # Extract the meta-classifier name
    meta_name = best_model_name.replace('Stacking (', '').replace(')', '')
    
    # Recreate the stacking classifier
    estimators = [(name, model) for name, model in tuned_models.items()]
    if meta_name in meta_classifiers:
        best_model = StackingClassifier(
            estimators=estimators,
            final_estimator=meta_classifiers[meta_name],
            cv=5,
            stack_method='predict_proba'
        )
        best_model.fit(X_train, y_train)
    else:
        print(f"Meta-classifier {meta_name} not found. Using first tuned model.")
        best_model = list(tuned_models.values())[0]
else:
    print(f"Model {best_model_name} not recognized. Using the first tuned model.")
    best_model = list(tuned_models.values())[0]

In [ ]:
# Feature importance analysis
# Method 1: Built-in feature importance (if available)
feature_importances = None

if hasattr(best_model, 'feature_importances_'):
    feature_importances = best_model.feature_importances_
elif hasattr(best_model, 'coef_'):
    # For linear models like Logistic Regression
    feature_importances = np.abs(best_model.coef_).mean(axis=0) if best_model.coef_.ndim > 1 else np.abs(best_model.coef_)

if feature_importances is not None:
    # Create DataFrame of feature importances
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importances
    })
    
    # Sort by importance
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    
    # Plot top 20 features
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=importance_df.head(20))
    plt.title(f'Top 20 Feature Importances for {best_model_name}', fontsize=16)
    plt.tight_layout()
    plt.show()
else:
    print("No built-in feature importance available for this model.")
    
    # Fallback: Use permutation importance
    print("Using permutation importance instead...")
    perm_importance = permutation_importance(best_model, X_test, y_test, 
                                            n_repeats=10, random_state=RANDOM_STATE)
    
    # Create DataFrame of permutation importances
    perm_importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': perm_importance.importances_mean
    })
    
    # Sort by importance
    perm_importance_df = perm_importance_df.sort_values(by='Importance', ascending=False)
    
    # Plot top 20 features
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=perm_importance_df.head(20))
    plt.title(f'Top 20 Permutation Importances for {best_model_name}', fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
# SHAP Analysis (if compatible with the model)
try:
    # Try to use SHAP for model interpretation
    # Note: This might not work for all model types
    explainer = shap.Explainer(best_model, X_train)
    shap_values = explainer(X_test.iloc[:100])  # Use a subset for computational efficiency
    
    # Summary plot
    plt.figure(figsize=(12, 10))
    shap.summary_plot(shap_values, X_test.iloc[:100], plot_type="bar")
    plt.title(f'SHAP Feature Importance for {best_model_name}', fontsize=16)
    plt.show()
    
    # SHAP value summary plot
    plt.figure(figsize=(12, 10))
    shap.summary_plot(shap_values, X_test.iloc[:100])
    plt.title(f'SHAP Summary Plot for {best_model_name}', fontsize=16)
    plt.show()
    
    # SHAP dependence plots for top features
    if feature_importances is not None:
        top_features = importance_df.head(3)['Feature'].tolist()
    else:
        top_features = perm_importance_df.head(3)['Feature'].tolist()
    
    for feature in top_features:
        plt.figure(figsize=(10, 6))
        shap.dependence_plot(feature, shap_values.values, X_test.iloc[:100], feature_names=feature_names)
        plt.title(f'SHAP Dependence Plot for {feature}', fontsize=16)
        plt.show()
except Exception as e:
    print(f"SHAP analysis failed: {e}")
    print("SHAP may not be compatible with this model type or structure.")

In [ ]:
# Analyze specific examples
# Let's look at a few examples from each class

# Get predictions from best model
y_pred_best = best_model.predict(X_test)
if hasattr(best_model, "predict_proba"):
    y_pred_proba_best = best_model.predict_proba(X_test)
else:
    y_pred_proba_best = None

# Function to find interesting examples
def find_interesting_examples(y_true, y_pred, proba=None, n_per_category=2):
    results = []
    
    # Correct predictions with high confidence
    correct = y_true == y_pred
    correct_indices = np.where(correct)[0]
    
    # Incorrect predictions
    incorrect = y_true != y_pred
    incorrect_indices = np.where(incorrect)[0]
    
    # For each true class
    for cls in np.unique(y_true):
        cls_indices = np.where(y_true == cls)[0]
        
        # Correct predictions for this class
        cls_correct = np.intersect1d(cls_indices, correct_indices)
        
        # If proba is available, find highest confidence
        if proba is not None and len(cls_correct) > 0:
            # Get confidence scores for the predicted class
            confidence = [proba[i][y_pred[i]] for i in cls_correct]
            # Sort by confidence (highest first)
            sorted_indices = cls_correct[np.argsort(-np.array(confidence))]
            # Take top n examples
            high_conf_correct = sorted_indices[:n_per_category] if len(sorted_indices) >= n_per_category else sorted_indices
        else:
            # Just take the first n examples if no probability available
            high_conf_correct = cls_correct[:n_per_category] if len(cls_correct) >= n_per_category else cls_correct
        
        for idx in high_conf_correct:
            confidence_val = proba[idx][y_pred[idx]] if proba is not None else None
            results.append({
                'Index': idx,
                'True Class': cls,
                'Predicted Class': y_pred[idx],
                'Confidence': confidence_val,
                'Correct': True,
                'Category': f"High confidence correct prediction for class {cls}"
            })
        
        # Incorrect predictions for this class
        cls_incorrect = np.intersect1d(cls_indices, incorrect_indices)
        
        if len(cls_incorrect) > 0:
            for idx in cls_incorrect[:n_per_category]:
                confidence_val = proba[idx][y_pred[idx]] if proba is not None else None
                results.append({
                    'Index': idx,
                    'True Class': cls,
                    'Predicted Class': y_pred[idx],
                    'Confidence': confidence_val,
                    'Correct': False,
                    'Category': f"Incorrect prediction for class {cls}"
                })
    
    return pd.DataFrame(results)

# Find interesting examples
examples = find_interesting_examples(y_test, y_pred_best, y_pred_proba_best)

# Map encoded class values back to original class names
examples['True Class Name'] = examples['True Class'].apply(lambda x: label_encoder.inverse_transform([x])[0])
examples['Predicted Class Name'] = examples['Predicted Class'].apply(lambda x: label_encoder.inverse_transform([x])[0])

# Display examples
display(examples)

## 9. Final Model Selection

Based on the comprehensive evaluation, let's select the final model for deployment.

In [ ]:
# Summarize all model performances
print("Model Performance Summary:")
display(all_model_results.sort_values(by='F1 Score (Macro)', ascending=False).head(10))

# Select final model
final_model = best_model
final_model_name = best_model_name

print(f"\nSelected Final Model: {final_model_name}")
print(f"F1 Score (Macro): {best_overall['F1 Score (Macro)']:.4f}")
print(f"Accuracy: {best_overall['Accuracy']:.4f}")

# Get predictions from final model
final_predictions = best_model.predict(X_test)
if hasattr(best_model, "predict_proba"):
    final_probabilities = best_model.predict_proba(X_test)
else:
    final_probabilities = None

# Print classification report for final model
print("\nClassification Report for Final Model:")
print(classification_report(y_test, final_predictions, target_names=class_names))

# Plot confusion matrix for final model
plot_confusion_matrix(y_test, final_predictions, class_names=class_names, 
                     title=f"Confusion Matrix for Final Model ({final_model_name})")

## 10. Model Deployment Preparation

Let's prepare the final model for deployment by saving it to disk.

In [ ]:
# Save the final model
model_output_path = f"{models_dir}/final_model.pkl"
with open(model_output_path, 'wb') as f:
    pickle.dump(final_model, f)
print(f"Final model saved to {model_output_path}")

# Save model metadata
model_metadata = {
    'model_name': final_model_name,
    'creation_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'performance': {
        'accuracy': float(best_overall['Accuracy']),
        'precision_macro': float(best_overall['Precision (Macro)']),
        'recall_macro': float(best_overall['Recall (Macro)']),
        'f1_macro': float(best_overall['F1 Score (Macro)']),
        'roc_auc': float(best_overall['ROC-AUC (OvR)']) if best_overall['ROC-AUC (OvR)'] is not None else None
    },
    'feature_names': feature_names,
    'target_mapping': {str(k): int(v) for k, v in target_mapping.items()},
    'class_names': class_names,
    'model_type': type(final_model).__name__
}

metadata_output_path = f"{models_dir}/model_metadata.json"
import json
with open(metadata_output_path, 'w') as f:
    json.dump(model_metadata, f, indent=4)
print(f"Model metadata saved to {metadata_output_path}")

In [ ]:
# Create a simplified prediction function for deployment
def predict_dropout_risk(data, model=final_model, encoder=label_encoder, feature_list=feature_names):
    """
    Make academic status predictions with the trained model.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        Data containing the required features for prediction
    model : trained model
        The trained classifier model
    encoder : LabelEncoder
        Encoder used to transform class labels
    feature_list : list
        List of features expected by the model
        
    Returns:
    --------
    dict containing:
        - predicted_class: The predicted academic status
        - probabilities: Probability scores for each class (if available)
    """
    # Ensure all required features are present
    missing_features = set(feature_list) - set(data.columns)
    if missing_features:
        raise ValueError(f"Missing required features: {missing_features}")
    
    # Extract features in the correct order
    X = data[feature_list]
    
    # Make predictions
    predicted_class_encoded = model.predict(X)
    predicted_class = encoder.inverse_transform(predicted_class_encoded)
    
    # Get probability scores if available
    if hasattr(model, "predict_proba"):
        probabilities = model.predict_proba(X)
        class_probs = {encoder.inverse_transform([i])[0]: probabilities[0][i] 
                       for i in range(len(encoder.classes_))}
    else:
        class_probs = None
    
    return {
        'predicted_class': predicted_class[0],
        'probabilities': class_probs
    }

# Save the prediction function
prediction_function_path = f"{models_dir}/predict_function.py"
with open(prediction_function_path, 'w') as f:
    f.write(inspect.getsource(predict_dropout_risk))
print(f"Prediction function saved to {prediction_function_path}")

# Test the prediction function on a sample
sample_data = X_test.iloc[[0]]
sample_prediction = predict_dropout_risk(sample_data)
print("\nSample Prediction:")
print(f"Predicted Class: {sample_prediction['predicted_class']}")
if sample_prediction['probabilities']:
    print("Class Probabilities:")
    for cls, prob in sample_prediction['probabilities'].items():
        print(f"  {cls}: {prob:.4f}")

## Summary and Next Steps

In this notebook, we performed comprehensive model experimentation for our academic status and dropout prediction system. Here's a summary of what we accomplished:

1. **Model Evaluation**:
   - Implemented and evaluated multiple classification algorithms
   - Compared model performance using accuracy, precision, recall, and F1 score
   - Validated models using cross-validation

2. **Advanced Techniques**:
   - Tuned hyperparameters for top-performing models
   - Explored ensemble methods including voting and stacking
   - Achieved improved performance through model optimization

3. **Model Interpretability**:
   - Analyzed feature importance to understand key predictors
   - Used SHAP values to interpret model decisions
   - Examined specific examples to gain insights into model behavior

4. **Deployment Preparation**:
   - Selected the best-performing model for deployment
   - Saved the model and its metadata for future use
   - Created a simplified prediction function for integration

**Next Steps**:
1. Integrate the model into a production pipeline
2. Develop a monitoring system to track model performance over time
3. Create a user interface for educational stakeholders
4. Implement feedback loops to improve model performance with new data

The final model demonstrates strong predictive performance for academic status and dropout risk, with important insights into the key factors influencing student outcomes. By deploying this model in an educational setting, institutions can identify at-risk students earlier and implement targeted interventions to improve retention and academic success.